In [ ]:
!pip install automata-lib gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggi

In [ ]:
# Import necessary libraries
from automata.fa.dfa import DFA
import gradio as gr

# Define the DFA with binary and alphabet characters
dfa = DFA(
    states={'q0', 'q1', 'q2', 'q3', 'q_ab'},  # Adding an extra state 'q_ab' for 'ab' detection
    input_symbols={'0', '1', 'a', 'b'},       # Extend input alphabet
    transitions={
        'q0': {'1': 'q1', '0': 'q0', 'a': 'q_ab', 'b': 'q0'},
        'q1': {'1': 'q2', '0': 'q0', 'a': 'q_ab', 'b': 'q0'},
        'q2': {'1': 'q2', '0': 'q3', 'a': 'q_ab', 'b': 'q0'},
        'q3': {'1': 'q3', '0': 'q3', 'a': 'q_ab', 'b': 'q0'},
        'q_ab': {'a': 'q_ab', 'b': 'q3', '0': 'q_ab', '1': 'q_ab'}
    },
    initial_state='q0',               # Initial state
    final_states={'q3', 'q_ab'}       # Accepting states for "110" and "ab" sequences
)

# Function to trace transitions and check acceptance
def check_string_with_trace(input_string):
    try:
        current_state = dfa.initial_state
        transitions = [f"Initial State: {current_state}"]

        # Process each character and record transitions with arrows
        for symbol in input_string:
            if symbol not in dfa.input_symbols:
                return "Error: Invalid character in input string."
            next_state = dfa.transitions[current_state][symbol]
            transitions.append(f"Input '{symbol}': {current_state} --'{symbol}'→ {next_state}")
            current_state = next_state

        # Final state and result
        result = "Accepted" if current_state in dfa.final_states else "Rejected"
        transitions.append(f"Final State: {current_state}")
        transitions.append(f"Result: {result}")
        return "\n".join(transitions)
    except Exception as e:
        return f"Error: {str(e)}"

# Options for pre-defined test cases
def example_string(option):
    examples = {
        "Ends with '11'": "111",
        "Contains 'ab'": "ab110",
        "Contains '1101'": "1101"
    }
    return examples.get(option, "")

# Gradio Blocks Interface
with gr.Blocks() as demo:
    gr.Markdown("## DFA for '110' and 'ab' Substrings with Transition Trace")
    gr.Markdown("Enter a binary or alphabet string to see if it contains '110' or 'ab' and view the transition trace. "
                "You can also choose a predefined example.")

    # Input components
    input_text = gr.Textbox(label="Enter string")
    dropdown = gr.Dropdown(choices=["Ends with '11'", "Contains 'ab'", "Contains '1101'"],
                           label="Choose a test case")

    # Clear input text when a dropdown item is selected
    def clear_input(choice):
        return ""

    dropdown.change(fn=clear_input, inputs=dropdown, outputs=input_text)

    # Output component
    output_text = gr.Textbox(label="Transition Trace")

    # Button to process DFA on input text
    submit_btn = gr.Button("Check String")
    submit_btn.click(fn=check_string_with_trace, inputs=input_text, outputs=output_text)

# Launch the interface
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://caef767d1c32915a5a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
